In [1]:
# rnn -> (batch, timestep, vector, pixel) 로 구성되어 있음

In [2]:
from __future__ import print_function

import keras
from keras.datasets import mnist
from keras.models import Model
from keras.layers import Input, Dense, TimeDistributed
from keras.layers import LSTM, SimpleRNN

In [3]:
batchSize = 32
numClasses = 10
epochs = 5

# embedding dims
rowHidden = 128
colHidden = 128

(xTrain, yTrain),(xTest, yTest) = mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [4]:
xTrain = xTrain.reshape(xTrain.shape[0],28,28,1) # data # sequence, vector, pixel 순서
xTest = xTest.reshape(xTest.shape[0], 28,28,1)
xTrain = xTrain.astype('float32')
xTest = xTest.astype('float32')
xTrain /= 255
xTest /= 255

print("xTrain shape : ",xTrain.shape)
print(xTrain.shape[0], "train samples")
print(xTest.shape[0], "test samples")

xTrain shape :  (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [5]:
yTrain = keras.utils.to_categorical(yTrain, numClasses)
yTest = keras.utils.to_categorical(yTest, numClasses)

In [6]:
vector, dim, pixel = xTrain.shape[1:] # 60000, 28, 28, 1 에서 60000 빼고?
x = Input(shape=(vector, dim, pixel)) # none, 28, 28, 1 인 셈, 앞에 나중에 batch size 추가

In [7]:
encodedRows = TimeDistributed(SimpleRNN(rowHidden))(x) # x = none, 28, 28, 1

In [8]:
encodedCols = SimpleRNN(colHidden)(encodedRows) 

In [9]:
prediction = Dense(10,activation = 'softmax')(encodedCols)
model = Model(inputs = x, outputs = prediction)
model.compile(loss='categorical_crossentropy', optimizer = 'rmsprop',metrics = ['accuracy'])

In [10]:
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
time_distributed (TimeDistri (None, 28, 128)           16640     
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 128)               32896     
_________________________________________________________________
dense (Dense)                (None, 10)                1290      
Total params: 50,826
Trainable params: 50,826
Non-trainable params: 0
_________________________________________________________________


In [11]:
model.fit(xTrain, yTrain, batch_size=batchSize, epochs = epochs, verbose = 1, validation_data= (xTest, yTest))

Epoch 1/5
1875/1875 [==============================] - 43s 23ms/step - loss: 0.5317 - accuracy: 0.8350 - val_loss: 0.2299 - val_accuracy: 0.9333
Epoch 2/5
1875/1875 [==============================] - 43s 23ms/step - loss: 0.4366 - accuracy: 0.8631 - val_loss: 0.2278 - val_accuracy: 0.9360
Epoch 3/5
1875/1875 [==============================] - 43s 23ms/step - loss: 0.5014 - accuracy: 0.8460 - val_loss: 0.2949 - val_accuracy: 0.9128
Epoch 4/5
1875/1875 [==============================] - 43s 23ms/step - loss: 0.2328 - accuracy: 0.9330 - val_loss: 0.1762 - val_accuracy: 0.9499
Epoch 5/5
1875/1875 [==============================] - 44s 24ms/step - loss: 0.2493 - accuracy: 0.9305 - val_loss: 0.1531 - val_accuracy: 0.9597


In [12]:
scores = model.evaluate(xTest, yTest, verbose = 0)
print("test loss : ", scores[0])
print("test acc : ", scores[1])

test loss :  0.15311308205127716
test acc :  0.9596999883651733
